# OSM Data Cleaning

In [1]:
# imports 
import pandas as pd

## Perform data cleaning & start exploration

In [2]:
slofood_parquet = pd.read_parquet('slo_food.parquet')
slodining_parquet = pd.read_parquet('slo_dining.parquet')
davisfood_parquet = pd.read_parquet('davis_food.parquet')
davisdining_parquet = pd.read_parquet('davis_dining.parquet')
athensfood_parquet = pd.read_parquet('athens_food.parquet')
athensdining_parquet = pd.read_parquet('athens_dining.parquet')

slofood_parquet.head()

,id,timestamp,lat,lon,amenity,brand,cuisine,name,shop
0,573859956,2024-03-19 13:03:23+00:00,35.249339,-120.642032,fast_food,Jersey Mike's Subs,sandwich,Jersey Mike's Subs,None
1,573864452,2024-06-20 01:42:44+00:00,35.250500,-120.642315,cafe,Starbucks,coffee_shop,Starbucks,None
2,574262633,2024-05-10 23:40:16+00:00,35.249171,-120.643033,cafe,Starbucks,coffee_shop,Starbucks,None
3,574262980,2024-05-10 21:22:16+00:00,35.250088,-120.643591,fast_food,Taco Bell,tex-mex,Taco Bell,None
4,581314991,2013-05-08 21:31:51+00:00,35.272740,-120.668584,None,None,None,Farb's Bakery,bakery


In [3]:
# check data type
print(slofood_parquet.dtypes)
print(davisfood_parquet.dtypes)
print(athensfood_parquet.dtypes)

id                        object
timestamp    datetime64[ns, UTC]
lat                      float64
lon                      float64
amenity                   object
brand                     object
cuisine                   object
name                      object
shop                      object
dtype: object
id                        object
timestamp    datetime64[ns, UTC]
lat                      float64
lon                      float64
amenity                   object
brand                     object
cuisine                   object
name                      object
shop                      object
dtype: object
id                        object
timestamp    datetime64[ns, UTC]
lat                      float64
lon                      float64
amenity                   object
name                      object
shop                      object
cuisine                   object
brand                     object
dtype: object


^^ This should be fine. Everything besides timestamp, latitude, and longitude can be a string (object). 

In [4]:
# check nulls (can keep in dataset)

print(slofood_parquet.isna().sum())
print('\n', davisfood_parquet.isna().sum())
print('\n', athensfood_parquet.isna().sum())

id             0
timestamp      0
lat            0
lon            0
amenity       30
brand        259
cuisine      170
name          84
shop         269
dtype: int64

 id             0
timestamp      0
lat            0
lon            0
amenity       11
brand        123
cuisine       58
name           9
shop         163
dtype: int64

 id             0
timestamp      0
lat            0
lon            0
amenity       47
name          41
shop         288
cuisine      185
brand        283
dtype: int64


^^ Nulls in `amenity`, `brand`, `cuisine`, and `shop` make sense since a food place may not be inside on of its categories. For example, a local restuarant will not be in a big brand. I would not need to remove these missing values since I know they have tag elsewhere.  

* But I want to check if there are spots with no description, and 
* I want to look at why `name` can be null.

In [5]:
# check if there are spots where all of the rows are null

print(len(slofood_parquet[slofood_parquet[['amenity', 'brand', 'cuisine', 'shop']].isnull().all(axis=1)]))
print(len(davisfood_parquet[davisfood_parquet[['amenity', 'brand', 'cuisine', 'shop']].isnull().all(axis=1)]))
print(len(athensfood_parquet[athensfood_parquet[['amenity', 'brand', 'cuisine', 'shop']].isnull().all(axis=1)]))

0
0
0


In [6]:
# check rows where 'name' is NaN
slofood_parquet[slofood_parquet['name'].isna()]

,id,timestamp,lat,lon,amenity,brand,cuisine,name,shop
19,1542001431,2022-10-30 20:09:34+00:00,35.300251,-120.662176,vending_machine,None,None,None,None
31,2119523722,2018-07-30 03:17:46+00:00,35.390997,-120.609380,restaurant,None,None,None,None
37,2213049089,2019-03-08 07:07:51+00:00,35.289045,-120.664937,fast_food,None,None,None,None
44,2262092650,2013-04-10 21:57:32+00:00,35.293103,-120.674052,bbq,None,None,None,None
76,2456191971,2024-03-10 17:11:34+00:00,35.280040,-120.661687,fast_food,None,pizza,None,None
...,...,...,...,...,...,...,...,...,...
286,11916083274,2024-05-18 14:52:14+00:00,35.280724,-120.659657,vending_machine,None,None,None,None
287,11916083275,2024-05-18 14:52:14+00:00,35.280766,-120.659281,vending_machine,None,None,None,None
288,11930836866,2024-05-24 14:47:50+00:00,35.267077,-120.674566,vending_machine,None,None,None,None
289,11930863369,2024-05-24 14:47:50+00:00,35.263561,-120.677559,vending_machine,None,None,None,None


^^ Name is null for local restaurants (that don't have much information on them), and also I included amenties where it is just a BBQ Picnic Spot at parks and vending machines, where both clearly would not have a name.

In [7]:
# number of places for each town

print("Number food places in SLO:", len(slofood_parquet))
print("Number food places in Davis:", len(davisfood_parquet))
print("Number food places in Athens:", len(athensfood_parquet))

Number food places in SLO: 301
Number food places in Davis: 174
Number food places in Athens: 337


^^ Originally, I was looking at just SLO and Davis, however, Davis does not have many food places, so it might not be the best place to compare with SLO. I decided to add Athens, GA to the list, and it seems a little more comparable to SLO, both have over 300 food spots. 

I also want to look at how the values for `amenity`, `brand`, `cuisine`, `name`, and `shop` look. 

In [8]:
for col in ['amenity', 'brand', 'cuisine', 'name', 'shop']:
    if col in slofood_parquet.columns:
        print(f"Unique values in {col} (SLO):\n", slofood_parquet[col].dropna().unique())
        print("------")
    
    if col in davisfood_parquet.columns:
        print(f"Unique values in {col} (Davis):\n", davisfood_parquet[col].dropna().unique())
        print("------")

    if col in athensfood_parquet.columns:
        print(f"Unique values in {col} (Athens):\n", athensfood_parquet[col].dropna().unique())
        print("------")

Unique values in amenity (SLO):
 ['fast_food' 'cafe' 'restaurant' 'bar' 'vending_machine' 'pub' 'bbq'
 'ice_cream' 'biergarten']
------
Unique values in amenity (Davis):
 ['fast_food' 'vending_machine' 'cafe' 'pub' 'restaurant' 'ice_cream' 'bbq'
 'bar']
------
Unique values in amenity (Athens):
 ['restaurant' 'pub' 'bar' 'pharmacy' 'cafe' 'fast_food' 'ice_cream' 'bbq'
 'vending_machine']
------
Unique values in brand (SLO):
 ["Jersey Mike's Subs" 'Starbucks' 'Taco Bell' '7-Eleven' "Domino's"
 'Subway' 'Panda Express' 'Five Guys' 'Redbox' 'El Pollo Loco'
 'The Habit Burger Grill' 'Rocky Mountain Chocolate Factory' 'Papyrus'
 'Jamba' "See's Candies" 'Popeyes' 'Which Wich?' 'Round Table Pizza'
 'Blaze Pizza' 'Yogurtland' 'Costco' 'Insomnia Cookies'
 'Einstein Bros. Bagels']
------
Unique values in brand (Davis):
 ['Del Taco' 'KFC' "Peet's Coffee" 'Jack in the Box' 'Taco Bell' 'Chipotle'
 'Starbucks' "Noah's Bagels" 'Jamba' 'The Habit Burger Grill'
 'Panera Bread' 'Burger King' "Domino's" 

^^ Most of these values seem nice to work with and are easy to read. However, for `cuisine`, some values are repeated with different wordings. This includes: 
* values containing mutiple cuisines separated by `;` (ex's: 'burger;diner', 'hot_dog;pizza;sandwich', 'street_food;burger;mexican;american')
* some of these combined values contain cuisines that are unrelated (ex: 'pizza;wine;beer' where 'wine' and 'beer' are not cuisines)

Additionally, the `amenity`, `cuisine`, and `shop` variables includes values with underscores, which should be replaced with spaces for consistency with other variables.

In [9]:
for parquet in [slofood_parquet, davisfood_parquet, athensfood_parquet]:
    multi_cuisine_rows = parquet[parquet['cuisine'].str.contains(';', na=False)]
    print(multi_cuisine_rows[['name', 'cuisine']])

                              name                              cuisine
21                 Mustang Station                      pizza;wine;beer
110                  Cool Cat Cafe                         burger;diner
183              Costco Food Court               hot_dog;pizza;sandwich
189  Taqueria Santa Cruz Express 2                      mexican;burrito
241                     Raku Ramen                noodle;japanese;ramen
244                      Poke Chef                           sushi;poke
245                        Ziggy's           american;mexican;ice_cream
254                        Streats  street_food;burger;mexican;american
271                     Jay Bird's                     chicken;sandwich
277         Nautical Bean - Bunker                   coffee_shop;bistro
                                             name  \
16                               Burgers and Brew   
19                                      Make Fish   
21                              Woodstock's Pizza

Manually looking through all the combined cuisines, however, show that they do pertain to all those cuisines. For cleaning, split the cuisines up as a list and keep them like that so if I wanted to filter through and search a specific cuisine can be uneecessarily complex. So, another way to handle it is keeping it as a string and using commas as separation.

In [10]:
# cleaning `cuisine` and `amenity` variables

# replace '_' with ' ' 
for parquet in [slofood_parquet, davisfood_parquet, athensfood_parquet]: 
    parquet['cuisine'] = parquet['cuisine'].str.replace('_', ' ')
    parquet['amenity'] = parquet['amenity'].str.replace('_', ' ')
    parquet['shop'] = parquet['amenity'].str.replace('_', ' ')

# multi-value handling (split into list)
for parquet in [slofood_parquet, davisfood_parquet, athensfood_parquet]: 
    parquet['cuisine'] = parquet['cuisine'].str.replace(';', ', ')
    # parquet["cuisine"] = parquet["cuisine"].dropna().apply(lambda x: x.split(";") if ";" in x else [x]) # even if it's only one cuisine, make it a list for consistency

In [11]:
for col in ['amenity', 'cuisine', 'shop']:
    if col in slofood_parquet.columns:
        print(f"Unique values in {col} (SLO):\n", slofood_parquet[col].dropna().astype(str).unique())
        print("------")

    if col in davisfood_parquet.columns:
        print(f"Unique values in {col} (Davis):\n", davisfood_parquet[col].dropna().astype(str).unique())
        print("------")

    if col in athensfood_parquet.columns:
        print(f"Unique values in {col} (Athens):\n", athensfood_parquet[col].dropna().astype(str).unique())
        print("------")

Unique values in amenity (SLO):
 ['fast food' 'cafe' 'restaurant' 'bar' 'vending machine' 'pub' 'bbq'
 'ice cream' 'biergarten']
------
Unique values in amenity (Davis):
 ['fast food' 'vending machine' 'cafe' 'pub' 'restaurant' 'ice cream' 'bbq'
 'bar']
------
Unique values in amenity (Athens):
 ['restaurant' 'pub' 'bar' 'pharmacy' 'cafe' 'fast food' 'ice cream' 'bbq'
 'vending machine']
------
Unique values in cuisine (SLO):
 ['sandwich' 'coffee shop' 'tex-mex' 'sushi' 'cajun' 'thai' 'bagel'
 'mediterranean' 'pizza, wine, beer' 'pizza' 'irish' 'mexican' 'ramen'
 'chinese' 'burger' 'breakfast' 'japanese' 'italian' 'barbecue' 'asian'
 'juice' 'indian' 'crepe' 'american' 'french' 'international' 'german'
 'burger, diner' 'chicken' 'turkish' 'frozen yogurt' 'poke' 'bakery'
 'steak house' 'hot dog, pizza, sandwich' 'mexican, burrito' 'bubble tea'
 'noodle, japanese, ramen' 'sushi, poke' 'american, mexican, ice cream'
 'noodle' 'street food, burger, mexican, american' 'chicken, sandwich'
 '

This is what the final values in each parquet look like:

In [12]:
for col in ['amenity', 'brand', 'cuisine', 'name', 'shop']:
    if col in slofood_parquet.columns:
        print(f"Unique values in {col} (SLO):\n", slofood_parquet[col].dropna().astype(str).unique())
        print("------")

    if col in davisfood_parquet.columns:
        print(f"Unique values in {col} (Davis):\n", davisfood_parquet[col].dropna().astype(str).unique())
        print("------")

    if col in athensfood_parquet.columns:
        print(f"Unique values in {col} (Athens):\n", athensfood_parquet[col].dropna().astype(str).unique())
        print("------")

Unique values in amenity (SLO):
 ['fast food' 'cafe' 'restaurant' 'bar' 'vending machine' 'pub' 'bbq'
 'ice cream' 'biergarten']
------
Unique values in amenity (Davis):
 ['fast food' 'vending machine' 'cafe' 'pub' 'restaurant' 'ice cream' 'bbq'
 'bar']
------
Unique values in amenity (Athens):
 ['restaurant' 'pub' 'bar' 'pharmacy' 'cafe' 'fast food' 'ice cream' 'bbq'
 'vending machine']
------
Unique values in brand (SLO):
 ["Jersey Mike's Subs" 'Starbucks' 'Taco Bell' '7-Eleven' "Domino's"
 'Subway' 'Panda Express' 'Five Guys' 'Redbox' 'El Pollo Loco'
 'The Habit Burger Grill' 'Rocky Mountain Chocolate Factory' 'Papyrus'
 'Jamba' "See's Candies" 'Popeyes' 'Which Wich?' 'Round Table Pizza'
 'Blaze Pizza' 'Yogurtland' 'Costco' 'Insomnia Cookies'
 'Einstein Bros. Bagels']
------
Unique values in brand (Davis):
 ['Del Taco' 'KFC' "Peet's Coffee" 'Jack in the Box' 'Taco Bell' 'Chipotle'
 'Starbucks' "Noah's Bagels" 'Jamba' 'The Habit Burger Grill'
 'Panera Bread' 'Burger King' "Domino's" 

In [13]:
# save this as new parquet

slofood_parquet.to_parquet("slofood_cleaned.parquet", index=False)
davisfood_parquet.to_parquet("davisfood_cleaned.parquet", index=False)
athensfood_parquet.to_parquet("athensfood_cleaned.parquet", index=False)